## Data preparation and sanitization

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import seaborn as sb

from core.utils.preprocessing import df_to_xy

# Read and sanitize the data
df = pd.read_excel("../data/t00/data.xls")
df = df.dropna()


x, y = df_to_xy(df, fuse_risk=True, centered=False, min_target=5, max_target=190)

## Linear regression (continuous dataset)

#### Benchmark

In [2]:
from core.utils.benchmark import process_benchmarks, process_benchmark_cv, process_model
from core.models.deep import KerasModel, loss_tf, gain_tf
from core.models import EMModel, RandomModel
from core.models import ConservativeModel
from sklearn.linear_model import LinearRegression, LogisticRegression
from core.models import RandomModel
from sklearn.neural_network import MLPClassifier

Using TensorFlow backend.


**Keras Model**

In [3]:
from core.models.deep import KerasModel, loss_tf, gain_tf

**Featureless model (fixed value)**

In [4]:
from core.models import EMModel, RandomModel

**Random Forest**

In [5]:
from sklearn.neural_network import MLPClassifier

**Acceptance model**

**Baseline models**

In [6]:
from sklearn.svm import SVC, SVR
from sklearn.svm import SVC, SVR
from core.utils.benchmark import benchmark_functions
from core.models.metrics import cross_compute

benchmark_models = {
    "random": RandomModel(),
    "conservative": ConservativeModel(),
    "em": EMModel(),
    "svc": SVC(gamma='auto'),
    "svr": SVR(gamma='auto'),
    "linear_regressor": LinearRegression(copy_X=False),
#     "linear_regressor_mse": keras_linear_regression(loss="mse"),
}
results = process_benchmarks(benchmark_models, x, y.ravel(), augment_data=[None])

results_mean = {key: item.mean() for key, item in results.items()}
results_mean["human"] = {f.__name__:cross_compute(df.min_offer, df['prop'], f) for f in benchmark_functions}
results_std = {key: item.std() for key, item in results.items()}
results_df = pd.DataFrame(results_mean).T
results_df.sort_values("avg_loss_ratio", inplace=True)
results_df

,avg_loss,avg_loss_ratio,avg_win_loss,invariance,loss_sum,mse,rejection_ratio
em,26.897619,0.218055,23.998552,1.000000,963.000000,1505.321429,0.066825
human,30.754190,0.261509,18.603896,0.158561,5505.000000,2594.832402,0.139665
svc,31.802381,0.278457,18.163757,0.204394,1141.000000,2413.464286,0.172381
svr,60.675561,0.573985,24.079861,0.292413,2172.908760,5403.002656,0.496825
linear_regressor,63.598092,0.582759,15.189009,0.176444,2278.057009,6512.577005,0.535873
random,70.892513,0.633293,33.429470,0.162209,2537.528721,6860.186820,0.519206
conservative,115.215079,0.953658,115.215079,1.000000,4124.000000,14136.424603,0.000000
